# In-vivo evidence for increased tau deposition in temporal lobe epilepsy

### Content
1. Participants
1.  *Figure 1:* Mapping of tau uptake in TLE
1.  *Figure 2:* Network contextualization
1.  *Figure 3:* Relationship to clinical and cognitive variables
1. Supplementary figures


In [ ]:
# Libraries
import numpy as np
import pandas as pd
from brainstat.stats.SLM import SLM
from brainstat.stats.terms import MixedEffect, FixedEffect
from brainstat.datasets import fetch_mask, fetch_template_surface
from brainspace.mesh.mesh_io import read_surface
from brainspace.plotting import plot_hemispheres

# Load utilities functions from utils.py
from utils import *

### Load data

In [ ]:
osf_path='/Users/rcruces/Desktop/2025/2025_mica_pet/OSF_data'

# Load the data frame
df = pd.read_csv(f'{osf_path}/18F-MK6240_database.csv')

# Load processed 18F-MK6240 PET data | matrix:{vertices x subjects}
pet_raw = np.load(f'{osf_path}/18F-MK6240_surf-fsLR-32k_desc-GroupData_smooth-10mm_pvc-probGM_ref-cerebellarGM_trc-MK6240_pet.npy')

# Load the cortical thickness data | matrix:{vertices x subjects}
cth_raw = np.load(f'{osf_path}/18F-MK6240_surf-fsLR-32k_desc-GroupData_smooth-20mm_thickness.npy')

# Load fsLR-32 surface
fslr32k_lh, fslr32k_rh = fetch_template_surface("fslr32k", join=False)
fslr32k = fetch_template_surface("fslr32k", join=True)

# Load fsLR-32k middle wall mask
fslr32k_mask = fetch_mask("fslr32k")

### Sort the matrices into ipsilateral/contralateral relative to seizure focus

In [ ]:
# fsLR-32k length
n_64k = pet_raw.shape[1]
n_32k = int(n_64k/2)

# Flip R >> L 18F-mk6240
mk_ipsi, mk_contra = flip_mat(pet_raw[:,0:n_32k], pet_raw[:,n_32k:n_64k], df['lateralization'].values, flip='R')

# Flip R >> L Thickness
th_ipsi, th_contra = flip_mat(cth_raw[:,0:n_32k], cth_raw[:,n_32k:n_64k], df['lateralization'].values, flip='R')

# Merge ipsi and contra
tauMK6240 = np.concatenate((mk_ipsi, mk_contra), axis=1)
thickness = np.concatenate((th_ipsi, th_contra), axis=1)


## Participants

In [ ]:
df



------------------------
## **Figure 1:** Mapping of tau uptake in TLE
### A | Mean Tau 18F-MK6240 uptake by group

In [ ]:
# plot data
plot_ctx_groups(fslr32k_lh, fslr32k_rh, tauMK6240, df , color_range=(0.5, 2), Save=False, Col='inferno', mask=fslr32k_mask, scale=2)


### B.1 | Tau 18F-MK6240 uptake group difference

### T-values map

In [ ]:
# p>0.025 per tail CREATE the mixed effects model
slm = mem_groups(Data=tauMK6240, df=df, Surf=fslr32k, mask=fslr32k_mask, Cthr=0.01, mem=True,  Pcorr="rft")

# Plot t-values
plot_ctx_slm(fslr32k_lh, fslr32k_rh, slm, color_range=(-3,3), Str='mk p01', Save=False, Col="RdBu_r", mask=fslr32k_mask, scale=2 )


### P-values map

### B.2 | Mean SUVR values for significant regions in each hemisphere by group

### C.1 | Subject-Wise Distribution of Cortical MK6240 Uptake
### C.1.a | Ridgeplot of TLE subjects

In [ ]:
# Filter mk_matched for the 'Healthy' group
healthy_mk_matched = mk_matched[df['group'] == 'Patient', :]

# Apply the thr_sig mask to select the appropriate columns
filtered_mk_matched = healthy_mk_matched[:, mask]

# Now plot using the filtered data
plot_ridgeplot(filtered_mk_matched, matrix_df=df[df['group'] == 'Patient'], 
               title="MK6240 distribution in PX", 
               Range=(0.5, 2.2), Cmap='inferno', Vline=1.5)


### C.1.b | Ridgeplot of Healthy controls

In [ ]:
# Filter mk_matched for the 'Healthy' group
healthy_mk_matched = mk_matched[df['group'] == 'Healthy', :]

# Apply the thr_sig mask to select the appropriate columns
filtered_mk_matched = healthy_mk_matched[:, mask]

# Now plot using the filtered data
plot_ridgeplot(filtered_mk_matched, df[df['group'] == 'Healthy'], 
               title="MK6240 distribution in HC", 
               Range=(0.5, 2.2), Cmap='inferno', Vline=1.5)


### C.2   | Individual examples of of Cortical [$^{18}F$]-MK6240 Uptake

### C.2.a | TLE subjects

In [ ]:
# Drop values of df
df = df.reset_index(drop=True)

# Low mk6240
# AGE: 52yrs, DURATION  2yrs, ONSET: 50yrs | PX020_02
# AGE: 33yrs, DURATION 15yrs, ONSET: 18yrs | PX050_01

# high mk6240
# AGE: 45yrs, DURATION 38yrs, ONSET: 7yrs | PX052_01
# AGE: 25yrs, DURATION  8yrs, ONSET: 17yrs | PX060_02
mk_list = [mk_matched[df.index[df['id'] == 'PX020_02'][0],:],
    mk_matched[df.index[df['id'] == 'PX050_01'][0],:],
    mk_matched[df.index[df['id'] == 'PX052_01'][0],:],
    mk_matched[df.index[df['id'] == 'PX060_02'][0],:] ]

# Plot subjects
plot_hemispheres(surf_lh, surf_rh, array_name=mk_list, size=(900, 1000),
                         color_bar='bottom', zoom=1.25, embed_nb=True,
                         interactive=False, share='both',
                         nan_color=(0, 0, 0, 1), color_range=(0.5, 2),
                         cmap='inferno', transparent_bg=True, label_text=['PXlow1', 'PXlow2','PXhigh1', 'PXhigh2'],
                         screenshot=False, filename='/home/bic/rcruces/Desktop/mk6240_PX-subjects.png', scale=1)


### C.2.b | Healthy controls

# Subset of subjects HC
mk_list = [mk_matched[df.index[df['id'] == 'HC062_01'][0],:], 
    mk_matched[df.index[df['id'] == 'HC010_02'][0],:],
    mk_matched[df.index[df['id'] == 'HC057_02'][0],:],
    mk_matched[df.index[df['id'] == 'HC014_01'][0],:]
          ]

# Plot subjects
plot_hemispheres(surf_lh, surf_rh, array_name=mk_list, size=(900, 1000),
                         color_bar='bottom', zoom=1.25, embed_nb=True,
                         interactive=False, share='both',
                         nan_color=(0, 0, 0, 1), color_range=(0.5, 2.2),
                         cmap='inferno', transparent_bg=True, label_text=['HClow1', 'HClow2', 'HChigh1', 'HChigh2'],
                         screenshot=False, scale=1)


----------------
## **Figure 2:** Network contextualization
### A | Relationship Between Tau MK6240 Group Differences and Functional Connectome

### Mean functional connectome as adjacency matrix

### Mean functional connectome as network

### Tau MK6240 weighted by the functional connectome neighbours

### B | Relationship Between Tau MK6240 Group Differences and Structural Connectome

### Mean structural connectome as adjacency matrix

### Mean structural connectome as network

### Tau MK6240 weighted by the structural connectome neighbours

-----------------
## **Figure 3:** Relationship to clinical and cognitive variables

### A | Tau MK6240 SUVR and clinical relationships

### Effect of age

### Effect of duration

### B | Tau MK6240 SUVR and behavioral relationships

### Epitrack score

### Episodic memory

### Semantic memory

---------------------
# Supplementary figures

## Cortical map of abnormal MK6240 SUVR probability

In [ ]:
# Create a new matrix with values higher than 1.5 set to 1, rest set to 0
mk_bin = np.where(mk_matched > 1.5, 1, 0)

# plot data
plot_ctx_groups(surf_lh, surf_rh, mk_bin, df , color_range=(-0.75,0.75), 
                Save=False, Col='RdBu_r', mask=mask_32k, scale=2)


## Group difference in cortical thickness

## Group difference in MK6240 controlled by thickness

## Mean longitudinal MK6240 by subject per group